In [1]:
#First Model

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

#To read the test set use this: df2 = pd.read_csv('readonly/test.csv')

df1 = pd.read_csv('train.csv', encoding = "ISO-8859-1")

dfLatLon = pd.read_csv('readonly/latlons.csv', encoding = 'ISO-8859-1')
dfAdress = pd.read_csv('readonly/addresses.csv', encoding = 'ISO-8859-1')

suspVariables = ['violation_zip_code','mailing_address_str_name',
                 'zip_code',
                 'country', 'clean_up_cost',
                 'ticket_issued_date','non_us_str_code',
                 'hearing_date', 'violation_description',
                 'admin_fee',
                 'state_fee','judgment_amount','payment_amount'
                 ,'payment_date','grafitti_status','payment_status',
                'collection_status','compliance_detail']

df1 = df1[df1['country'] == 'USA']
df1.drop(suspVariables, axis = 1, inplace = True)

dfAdress = dfAdress.merge(dfLatLon, how = 'inner', on = 'address')
df1 = df1.merge(dfAdress, how = 'inner', on = 'ticket_id')

df1 = df1[df1['compliance'].isnull() != True].dropna()

#get dummy variables and get rid of the rest:

dfdummies = pd.get_dummies(df1,columns=['violation_code'])

dfdummies = dfdummies.sample(n=10000)

X = dfdummies.select_dtypes(include=['uint8']).copy()
X = X.drop(['violation_code_19420901'],axis=1)
y = dfdummies['compliance']

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X,y)

df2 = pd.read_csv('readonly/test.csv')
df2 = pd.get_dummies(df2,columns=['violation_code'])
X_test = df2.select_dtypes(include=['uint8']).copy()
X_test = X_test.reindex(columns = X.columns, fill_value = 0)


knn.score(X,y)
#The following instruction was found in:
#https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data

X_test = X_test.reindex(columns = X.columns, fill_value = 0)
prob = knn.predict_proba(X_test)

probList = []
for i in prob:
    probList.append(i[0])

pd.Series(data = probList, index = df2['ticket_id'])

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
def  blight_model():
    import pandas as pd
    import numpy as np
    from sklearn.neighbors import KNeighborsClassifier

    #To read the test set use this: df2 = pd.read_csv('readonly/test.csv')

    df1 = pd.read_csv('train.csv', encoding = "ISO-8859-1")

    dfLatLon = pd.read_csv('readonly/latlons.csv', encoding = 'ISO-8859-1')
    dfAdress = pd.read_csv('readonly/addresses.csv', encoding = 'ISO-8859-1')

    suspVariables = ['violation_zip_code','mailing_address_str_name',
                     'zip_code',
                     'country', 'clean_up_cost',
                     'ticket_issued_date','non_us_str_code',
                     'hearing_date', 'violation_description',
                     'admin_fee',
                     'state_fee','judgment_amount','payment_amount'
                     ,'payment_date','grafitti_status','payment_status',
                    'collection_status','compliance_detail']

    df1 = df1[df1['country'] == 'USA']
    df1.drop(suspVariables, axis = 1, inplace = True)

    dfAdress = dfAdress.merge(dfLatLon, how = 'inner', on = 'address')
    df1 = df1.merge(dfAdress, how = 'inner', on = 'ticket_id')

    df1 = df1[df1['compliance'].isnull() != True].dropna()

    #get dummy variables and get rid of the rest:

    dfdummies = pd.get_dummies(df1,columns=['violation_code'])

    dfdummies = dfdummies.sample(n=10000)

    X = dfdummies.select_dtypes(include=['uint8']).copy()
    X = X.drop(['violation_code_19420901'],axis=1)
    y = dfdummies['compliance']

    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(X,y)

    df2 = pd.read_csv('readonly/test.csv')
    df2 = pd.get_dummies(df2,columns=['violation_code'])
    X_test = df2.select_dtypes(include=['uint8']).copy()
    X_test = X_test.reindex(columns = X.columns, fill_value = 0)


    knn.score(X,y)

    X_test = X_test.reindex(columns = X.columns, fill_value = 0)
    prob = knn.predict_proba(X_test)

    probList = []
    for i in prob:
        probList.append(i[0])

    return pd.Series(data = probList, index = df2['ticket_id'])

probSeries = blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [1]:
#Second Model:

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

#To read the test set use this: df2 = pd.read_csv('readonly/test.csv')

df1 = pd.read_csv('train.csv', encoding = "ISO-8859-1")

dfLatLon = pd.read_csv('readonly/latlons.csv', encoding = 'ISO-8859-1')
dfAdress = pd.read_csv('readonly/addresses.csv', encoding = 'ISO-8859-1')

df2 = pd.read_csv('readonly/test.csv')

varsToGetRidOf = []

for i in df1.columns:
    if not i in df2 and i != 'compliance':
        varsToGetRidOf.append(i)
        
df1 =df1.drop(varsToGetRidOf, axis = 1)

df1 = df1[df1['compliance'].isnull() != True]

df1 = df1[df1['country'] == 'USA']

VarsToKeep = ['ticket_id','agency_name','inspector_name','state', 'ticket_issued_date', 'hearing_date',
            'disposition','fine_amount','late_fee','discount_amount','judgment_amount','compliance']

df1 = df1[VarsToKeep]

dfAdress = dfAdress.merge(dfLatLon, how = 'inner', on = 'address')
df1 = df1.merge(dfAdress, how = 'inner', on = 'ticket_id').drop('address',axis = 1)

df1Dummies = pd.get_dummies(df1, columns = ['disposition'])

df1Dummies['hearing_date'] = pd.to_datetime(df1Dummies['hearing_date']).apply(lambda x: x.month)
df1Dummies['ticket_issued_date'] = pd.to_datetime(df1Dummies['ticket_issued_date']).apply(lambda x: x.month)

df1Dummies = pd.get_dummies(df1Dummies, columns = ['ticket_issued_date','hearing_date','agency_name','state'])

df1Dummies.drop('inspector_name', axis = 1, inplace=True)

df1Dummies = df1Dummies.reset_index().iloc[:,1:].set_index('ticket_id')
df1Dummies.index.name = None
df1Dummies = df1Dummies.dropna()

#Creating X and y train and test sets:

X = df1Dummies.drop('compliance', axis = 1)
y = df1Dummies['compliance']

#Creating model:

clf = RandomForestClassifier(max_features = 30).fit(X,y)

# Processing X_test set:

VarsToKeep2 = ['ticket_id','agency_name','inspector_name','state', 'ticket_issued_date', 'hearing_date',
            'disposition','fine_amount','late_fee','discount_amount','judgment_amount']

X_testset = df2[VarsToKeep2]

X_testset = X_testset.merge(dfAdress, how = 'inner', on = 'ticket_id').drop('address',axis = 1)

XtsDummies = pd.get_dummies(X_testset, columns = ['disposition'])

XtsDummies['hearing_date'] = pd.to_datetime(XtsDummies['hearing_date']).apply(lambda x: x.month)
XtsDummies['ticket_issued_date'] = pd.to_datetime(XtsDummies['ticket_issued_date']).apply(lambda x: x.month)

XtsDummies = pd.get_dummies(XtsDummies, columns = ['ticket_issued_date','hearing_date','agency_name','state'])

#Replacing nas with the mean

clMeans = XtsDummies.mean()
XtsDummies = XtsDummies.fillna(clMeans)

XtsDummies = XtsDummies.reset_index().iloc[:,1:].set_index('ticket_id')

XtsDummies.drop('inspector_name', axis = 1, inplace=True)

XtsDummies = XtsDummies.reindex(columns = X.columns, fill_value = 0)

#calculating probs:

probArray = clf.predict_proba(XtsDummies)

probList = [i[1] for i in probArray]

#creating pandas series

toReturn = pd.Series(probList, index = XtsDummies.index)


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
#Second Model:

def  blight_model():

    import pandas as pd
    import numpy as np
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split

    #To read the test set use this: df2 = pd.read_csv('readonly/test.csv')

    df1 = pd.read_csv('train.csv', encoding = "ISO-8859-1")

    dfLatLon = pd.read_csv('readonly/latlons.csv', encoding = 'ISO-8859-1')
    dfAdress = pd.read_csv('readonly/addresses.csv', encoding = 'ISO-8859-1')

    df2 = pd.read_csv('readonly/test.csv')

    varsToGetRidOf = []

    for i in df1.columns:
        if not i in df2 and i != 'compliance':
            varsToGetRidOf.append(i)

    df1 =df1.drop(varsToGetRidOf, axis = 1)

    df1 = df1[df1['compliance'].isnull() != True]

    df1 = df1[df1['country'] == 'USA']

    VarsToKeep = ['ticket_id','agency_name','inspector_name','state', 'ticket_issued_date', 'hearing_date',
                'disposition','fine_amount','late_fee','discount_amount','judgment_amount','compliance']

    df1 = df1[VarsToKeep]

    dfAdress = dfAdress.merge(dfLatLon, how = 'inner', on = 'address')
    df1 = df1.merge(dfAdress, how = 'inner', on = 'ticket_id').drop('address',axis = 1)

    df1Dummies = pd.get_dummies(df1, columns = ['disposition'])

    df1Dummies['hearing_date'] = pd.to_datetime(df1Dummies['hearing_date']).apply(lambda x: x.month)
    df1Dummies['ticket_issued_date'] = pd.to_datetime(df1Dummies['ticket_issued_date']).apply(lambda x: x.month)

    df1Dummies = pd.get_dummies(df1Dummies, columns = ['ticket_issued_date','hearing_date','agency_name','state'])

    df1Dummies.drop('inspector_name', axis = 1, inplace=True)

    df1Dummies = df1Dummies.reset_index().iloc[:,1:].set_index('ticket_id')
    df1Dummies.index.name = None
    df1Dummies = df1Dummies.dropna()

    #Creating X and y train and test sets:

    X = df1Dummies.drop('compliance', axis = 1)
    y = df1Dummies['compliance']

    #Creating model:

    clf = RandomForestClassifier(max_features = 30).fit(X,y)

    # Processing X_test set:

    VarsToKeep2 = ['ticket_id','agency_name','inspector_name','state', 'ticket_issued_date', 'hearing_date',
                'disposition','fine_amount','late_fee','discount_amount','judgment_amount']

    X_testset = df2[VarsToKeep2]

    X_testset = X_testset.merge(dfAdress, how = 'inner', on = 'ticket_id').drop('address',axis = 1)

    XtsDummies = pd.get_dummies(X_testset, columns = ['disposition'])

    XtsDummies['hearing_date'] = pd.to_datetime(XtsDummies['hearing_date']).apply(lambda x: x.month)
    XtsDummies['ticket_issued_date'] = pd.to_datetime(XtsDummies['ticket_issued_date']).apply(lambda x: x.month)

    XtsDummies = pd.get_dummies(XtsDummies, columns = ['ticket_issued_date','hearing_date','agency_name','state'])

    #Replacing nas with the mean

    clMeans = XtsDummies.mean()
    XtsDummies = XtsDummies.fillna(clMeans)

    XtsDummies = XtsDummies.reset_index().iloc[:,1:].set_index('ticket_id')

    XtsDummies.drop('inspector_name', axis = 1, inplace=True)

    XtsDummies = XtsDummies.reindex(columns = X.columns, fill_value = 0)

    #calculating probs:

    probArray = clf.predict_proba(XtsDummies)

    probList = [i[1] for i in probArray]

    #creating pandas series

    return pd.Series(probList, index = XtsDummies.index)


In [2]:
toReturn

ticket_id
284932    0.0
285362    0.0
285361    0.1
285338    0.0
285346    0.1
285345    0.0
285347    0.2
285342    1.0
285530    0.0
284989    0.0
285344    0.1
285343    0.0
285340    0.0
285341    0.2
285349    0.0
285348    0.0
284991    0.0
285532    0.0
285406    0.0
285001    0.0
285006    0.0
285405    0.0
285337    0.0
285496    0.1
285497    0.0
285378    0.0
285589    0.0
285585    0.0
285501    0.0
285581    0.0
         ... 
376367    0.0
376366    0.0
376362    0.5
376363    0.2
376365    0.0
376364    0.0
376228    0.1
376265    0.0
376286    0.3
376320    0.1
376314    0.0
376327    0.6
376385    0.1
376435    1.0
376370    1.0
376434    0.2
376459    0.0
376478    0.0
376473    0.0
376484    0.0
376482    0.1
376480    0.1
376479    0.1
376481    0.1
376483    0.0
376496    0.0
376497    0.0
376499    0.0
376500    0.0
369851    0.7
dtype: float64

In [26]:
for i in probarray:
    print(i[0])
        

1.0
1.0
1.0
1.0
0.9
0.9
0.9
0.0
1.0
1.0
0.9
1.0
1.0
0.8
0.9
1.0
1.0
1.0
1.0
1.0
1.0
0.9
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9
1.0
1.0
1.0
1.0
1.0
0.0
0.6
0.8
0.7
0.5
1.0
1.0
1.0
1.0
0.3
1.0
1.0
1.0
0.9
1.0
1.0
1.0
0.9
0.9
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.1
1.0
0.9
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9
1.0
0.7
0.9
1.0
1.0
1.0
0.8
0.2
0.0
0.0
1.0
1.0
0.8
1.0
1.0
1.0
0.9
1.0
1.0
0.9
1.0
0.8
1.0
0.9
1.0
0.9
0.9
1.0
1.0
0.9
1.0
1.0
1.0
0.8
1.0
1.0
1.0
1.0
1.0
1.0
0.8
1.0
1.0
1.0
0.9
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.9
1.0
0.9
1.0
1.0
0.2
0.5
1.0
1.0
1.0
1.0
1.0
0.9
1.0
1.0
0.9
1.0
1.0
1.0
1.0
0.9
0.9
1.0
0.8
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9
1.0
1.0
1.0
1.0
1.0
0.8
0.8
1.0
0.0
1.0
1.0
1.0
0.9
1.0
0.9
0.9
1.0
1.0
0.8
0.8
1.0
1.0
1.0
0.9
1.0
0.9
1.0
0.1
1.0
0.9
0.9
0.8
1.0
1.0
1.0
1.0
1.0
1.0
0.9
1.0
1.0
0.9
0.5
1.0
1.0
0.3
1.0
0.7
0.9
0.9
1.0
0.9
0.8
1.0
0.1
0.9
1.0
0.7
1.0
1.0
1.0
0.2
0.7
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9
1.0
1.0
1.0
0.9
0.9
1.0
0.0
0.0
1.0


In [32]:
len(df1Dummies[df1Dummies['compliance'] == 0])/len(df1Dummies)

0.927470960235696